In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import time
import json
import csv

In [ ]:
import pandas as pd
urls_df = pd.read_csv('./archive/UE-NY-rest-urls.csv')
urls_df.head()

In [ ]:
def get_rest_details(rest_url, error_rests):
    start_time = time.time()
    rest_obj = {}
    try:
        req = Request(rest_url, headers={'User-Agent': 'Mozilla/5.0', 'Accept': 'text/html,application/xhtml+xml,application/xml'})
        webpage = urlopen(req, timeout=10).read()
        soup = BeautifulSoup(webpage, 'html.parser')
        rest_obj = json.loads(soup.find("script", type="application/ld+json").text)
    except Exception as e:
        print(f"\nerror: {e}\n url: {rest_url}")
        error_rests.append(rest_url)
    elapsed_time = time.time()-start_time
    return rest_obj

In [ ]:
def scrap_all_rest(urls_df, f, error_rests):
    start_time = time.time()
    num_rests = urls_df.shape[0]
    writer = csv.writer(f, delimiter=',')
    for i in range(2613, num_rests):
        curr_rest = urls_df.iloc[i]
        if i % 10 == 0:
                print(f"Progress: {(i*100)//num_rests}%, restaurants done: {i}, time elapsed: {time.time()-start_time}")
                
        try:
            obj = get_rest_details(curr_rest.url, error_rests)
            writer.writerow([
                obj['name'].replace(';', ''),
                curr_rest.city,
                obj.get('servesCuisine'),
                obj.get('aggregateRating'),
                obj.get('priceRange'),
                obj.get('geo'),
                obj.get('telephone'),
                curr_rest.url,
                obj.get('address'),
                obj.get('openingHoursSpecification'),
            ])
        except Exception as e:
            print(f"\nerror: {e}\n url: {curr_rest.url}")
            error_rests.append(curr_rest.url)

    elapsed_time = time.time()-start_time
    print(f"total time: {elapsed_time}, error count: {len(error_rests)}")
    return error_rests

In [ ]:
f = open('./UE-NY-rest-details.csv', 'a')
error_rests = []
scrap_all_rest(urls_df, f, error_rests)
# curr_rest = urls_df.iloc[0]
# curr_rest.url